# Assignment 2: Building MLPs, CNNs, and Generative Models

## Task 1: Learn the basics of Keras API for Tensorflow

## Task 2: Develop a "Tell-the-time" network

In [1]:
import numpy as np
import matplotlib.pyplot as plt
from tensorflow import keras
from tensorflow.keras import layers
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow.keras import backend as K

In [2]:
K.clear_session()

In [3]:
data = np.load("./a2_data/images.npy")
label = np.load("./a2_data/labels.npy")

In [4]:
data = data/255.

Regression Task:

In [8]:
label_r=label[:,0]+label[:,1]/60

In [9]:
def common_sense_reg(y_true, y_pred):
    d = K.abs(y_true-y_pred)
    return (1-(d//6))*(d)+(d//6)*(12-d)

In [11]:
X, X_test, y, y_test = train_test_split(data, label_r, test_size=0.2, random_state=42)
print(f"Train set: {X.shape}")
print(f"Test set: {X_test.shape}")

Train set: (14400, 150, 150)
Test set: (3600, 150, 150)


In [19]:
regression_model = keras.Sequential(
    [
        keras.Input(shape=(150,150,1)),
        layers.Conv2D(32, kernel_size=(3, 3), activation="relu",strides=1),
        layers.MaxPooling2D(pool_size=(2, 2)),
        layers.Conv2D(16, kernel_size=(3, 3), activation="relu",strides=2),
        layers.MaxPooling2D(pool_size=(2, 2)),
        layers.Conv2D(16, kernel_size=(3, 3), activation="relu",strides=1),
        layers.MaxPooling2D(pool_size=(2, 2)),
        layers.Flatten(),
        layers.Dense(64,activation="relu"),
        layers.Dense(1),
    ]
)

regression_model.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_6 (Conv2D)           (None, 148, 148, 32)      320       
                                                                 
 max_pooling2d_6 (MaxPooling  (None, 74, 74, 32)       0         
 2D)                                                             
                                                                 
 conv2d_7 (Conv2D)           (None, 36, 36, 16)        4624      
                                                                 
 max_pooling2d_7 (MaxPooling  (None, 18, 18, 16)       0         
 2D)                                                             
                                                                 
 conv2d_8 (Conv2D)           (None, 16, 16, 16)        2320      
                                                                 
 max_pooling2d_8 (MaxPooling  (None, 8, 8, 16)        

In [20]:
batch_size = 64
epochs = 60
regression_model.compile(loss="mse", optimizer="adam", metrics=common_sense_reg)

In [21]:
regression_model.fit(X,y, batch_size=batch_size, epochs=epochs, validation_split=0.1)

Epoch 1/60
203/203 [==============================] - 5s 19ms/step - loss: 13.7863 - common_sense_reg: 3.0049 - val_loss: 12.3277 - val_common_sense_reg: 3.0180
Epoch 2/60
203/203 [==============================] - 4s 18ms/step - loss: 12.1149 - common_sense_reg: 2.9987 - val_loss: 11.9883 - val_common_sense_reg: 2.9832
Epoch 3/60
203/203 [==============================] - 4s 18ms/step - loss: 11.7883 - common_sense_reg: 2.9071 - val_loss: 10.9861 - val_common_sense_reg: 2.7747
Epoch 4/60
203/203 [==============================] - 4s 18ms/step - loss: 10.5699 - common_sense_reg: 2.6683 - val_loss: 10.0905 - val_common_sense_reg: 2.5747
Epoch 5/60
203/203 [==============================] - 4s 17ms/step - loss: 9.7920 - common_sense_reg: 2.4883 - val_loss: 9.7146 - val_common_sense_reg: 2.4516
Epoch 6/60
203/203 [==============================] - 4s 18ms/step - loss: 9.4181 - common_sense_reg: 2.4291 - val_loss: 9.0243 - val_common_sense_reg: 2.3464
Epoch 7/60
203/203 [==================

In [22]:
regression_model.evaluate(X_test,y_test)

113/113 [==============================] - 0s 4ms/step - loss: 4.0751 - common_sense_reg: 1.4781


[4.0750956535339355, 1.4780513048171997]

Classfication Task

24 classes

In [14]:
import numpy as np
import matplotlib.pyplot as plt
from tensorflow import keras
from tensorflow.keras import layers
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow.keras import backend as K

In [15]:
y_c=[]
for i in range(24):
    la=np.zeros(24)
    la[i]=la[i]+1
    for j in range(750):
        y_c.append(la)

In [16]:
y_c=np.stack(y_c)
y_c=np.array(y_c,dtype=np.int32)

In [17]:
y_c.shape

(18000, 24)

def common_sense_cla(y_true, y_pred):
    y_true = y_true.numpy().reshape(-1,)
    y_pred = y_pred.numpy()
    y_pred = y_pred.argmax(axis=1)
    d = np.abs(y_true-y_pred)
    return 0.5*((1-(d//12))*(d)+(d//12)*(24-d))

In [18]:
def common_sense_cla(y_true, y_pred):
    d = K.abs(K.argmax(y_true, axis=1) - K.argmax(y_pred, axis=1))
    d = tf.cast(d, tf.float32)
    return 0.5 * ((1 - (d // 12)) * (d) + (d // 12) * (24 - d))

In [19]:
X, X_test, y, y_test = train_test_split(data, y_c, test_size=0.2, random_state=42)
print(f"Train set: {X.shape}")
print(f"Test set: {X_test.shape}")

Train set: (14400, 150, 150)
Test set: (3600, 150, 150)


In [20]:
classification_model = keras.Sequential(
    [
        keras.Input(shape=(150,150,1)),
        layers.Conv2D(16, kernel_size=(3, 3), activation="relu",strides=1),
        layers.MaxPooling2D(pool_size=(2, 2)),
        layers.Conv2D(32, kernel_size=(3, 3), activation="relu",strides=2),
        layers.MaxPooling2D(pool_size=(2, 2)),
        layers.Conv2D(64, kernel_size=(3, 3), activation="relu",strides=1),
        layers.MaxPooling2D(pool_size=(2, 2)),
        layers.Flatten(),
        layers.Dense(64,activation=keras.layers.LeakyReLU(alpha=0.01)),
        layers.Dense(24,activation="softmax"),
    ]
)

classification_model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_3 (Conv2D)           (None, 148, 148, 16)      160       
                                                                 
 max_pooling2d_3 (MaxPooling  (None, 74, 74, 16)       0         
 2D)                                                             
                                                                 
 conv2d_4 (Conv2D)           (None, 36, 36, 32)        4640      
                                                                 
 max_pooling2d_4 (MaxPooling  (None, 18, 18, 32)       0         
 2D)                                                             
                                                                 
 conv2d_5 (Conv2D)           (None, 16, 16, 64)        18496     
                                                                 
 max_pooling2d_5 (MaxPooling  (None, 8, 8, 64)        

In [21]:
batch_size = 128
epochs = 40
classification_model.compile(loss="categorical_crossentropy", optimizer=tf.keras.optimizers.Adam(learning_rate=0.001,), metrics=["accuracy",common_sense_cla], run_eagerly=True)

In [22]:
classification_model.fit(X,y, batch_size=batch_size, epochs=epochs, validation_split=0.1)

Epoch 1/40
102/102 [==============================] - 5s 46ms/step - loss: 3.1791 - accuracy: 0.0402 - common_sense_cla: 3.0238 - val_loss: 3.1787 - val_accuracy: 0.0333 - val_common_sense_cla: 3.0472
Epoch 2/40
102/102 [==============================] - 5s 44ms/step - loss: 3.1782 - accuracy: 0.0389 - common_sense_cla: 2.9904 - val_loss: 3.1785 - val_accuracy: 0.0444 - val_common_sense_cla: 3.0687
Epoch 3/40
102/102 [==============================] - 4s 43ms/step - loss: 3.1696 - accuracy: 0.0454 - common_sense_cla: 2.9644 - val_loss: 3.1186 - val_accuracy: 0.0681 - val_common_sense_cla: 2.6760
Epoch 4/40
102/102 [==============================] - 4s 43ms/step - loss: 2.8926 - accuracy: 0.1087 - common_sense_cla: 2.3547 - val_loss: 2.6603 - val_accuracy: 0.1417 - val_common_sense_cla: 2.0260
Epoch 5/40
102/102 [==============================] - 4s 43ms/step - loss: 2.4555 - accuracy: 0.1973 - common_sense_cla: 1.9017 - val_loss: 2.3536 - val_accuracy: 0.2208 - val_common_sense_cla: 1.

In [23]:
classification_model.evaluate(X_test, y_test)

113/113 [==============================] - 1s 11ms/step - loss: 1.9283 - accuracy: 0.6747 - common_sense_cla: 0.5114


[1.9283486604690552, 0.6747221946716309, 0.511388897895813]

72 classes

In [18]:
import numpy as np
import matplotlib.pyplot as plt
from tensorflow import keras
from tensorflow.keras import layers
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow.keras import backend as K

In [19]:
y_c=[]
for i in range(72):
    la=np.zeros(72)
    la[i]=la[i]+1
    for j in range(250):
        y_c.append(la)

In [20]:
y_c=np.stack(y_c)
y_c=np.array(y_c,dtype=np.int32)

In [21]:
y_c.shape

(18000, 72)

In [22]:
def common_sense_cla(y_true, y_pred):
    d = K.abs(K.argmax(y_true, axis=1) - K.argmax(y_pred, axis=1))
    d = tf.cast(d, tf.float32)
    return 0.5 * ((1 - (d // 36)) * (d) + (d // 36) * (72 - d))

In [23]:
X, X_test, y, y_test = train_test_split(data, y_c, test_size=0.2, random_state=42)
print(f"Train set: {X.shape}")
print(f"Test set: {X_test.shape}")

Train set: (14400, 150, 150)
Test set: (3600, 150, 150)


In [24]:
classification_model = keras.Sequential(
    [
        keras.Input(shape=(150,150,1)),
        layers.Conv2D(16, kernel_size=(3, 3), activation="relu",strides=1),
        layers.MaxPooling2D(pool_size=(2, 2)),
        layers.Conv2D(32, kernel_size=(3, 3), activation="relu",strides=2),
        layers.MaxPooling2D(pool_size=(2, 2)),
        layers.Conv2D(64, kernel_size=(3, 3), activation="relu",strides=1),
        layers.MaxPooling2D(pool_size=(2, 2)),
        layers.Flatten(),
        layers.Dense(64,activation=keras.layers.LeakyReLU(alpha=0.01)),
        layers.Dense(72,activation="softmax"),
    ]
)

classification_model.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_6 (Conv2D)           (None, 148, 148, 16)      160       
                                                                 
 max_pooling2d_6 (MaxPooling  (None, 74, 74, 16)       0         
 2D)                                                             
                                                                 
 conv2d_7 (Conv2D)           (None, 36, 36, 32)        4640      
                                                                 
 max_pooling2d_7 (MaxPooling  (None, 18, 18, 32)       0         
 2D)                                                             
                                                                 
 conv2d_8 (Conv2D)           (None, 16, 16, 64)        18496     
                                                                 
 max_pooling2d_8 (MaxPooling  (None, 8, 8, 64)        

In [25]:
batch_size = 128
epochs = 40
classification_model.compile(loss="categorical_crossentropy", optimizer=tf.keras.optimizers.Adam(learning_rate=0.001,), metrics=["accuracy",common_sense_cla], run_eagerly=True)

In [26]:
classification_model.fit(X,y, batch_size=batch_size, epochs=epochs, validation_split=0.1)

Epoch 1/40
102/102 [==============================] - 5s 48ms/step - loss: 4.2780 - accuracy: 0.0116 - common_sense_cla: 8.9613 - val_loss: 4.2771 - val_accuracy: 0.0104 - val_common_sense_cla: 9.0573
Epoch 2/40
102/102 [==============================] - 5s 45ms/step - loss: 4.2768 - accuracy: 0.0146 - common_sense_cla: 8.9604 - val_loss: 4.2775 - val_accuracy: 0.0090 - val_common_sense_cla: 9.0656
Epoch 3/40
102/102 [==============================] - 5s 45ms/step - loss: 4.2769 - accuracy: 0.0147 - common_sense_cla: 9.0147 - val_loss: 4.2773 - val_accuracy: 0.0160 - val_common_sense_cla: 8.8580
Epoch 4/40
102/102 [==============================] - 5s 45ms/step - loss: 4.2768 - accuracy: 0.0141 - common_sense_cla: 8.9408 - val_loss: 4.2781 - val_accuracy: 0.0090 - val_common_sense_cla: 9.0726
Epoch 5/40
102/102 [==============================] - 4s 43ms/step - loss: 4.2766 - accuracy: 0.0149 - common_sense_cla: 8.9634 - val_loss: 4.2782 - val_accuracy: 0.0090 - val_common_sense_cla: 9.

In [27]:
classification_model.evaluate(X_test, y_test)

113/113 [==============================] - 1s 10ms/step - loss: 2.0150 - accuracy: 0.6017 - common_sense_cla: 2.1254


[2.015038013458252, 0.6016666889190674, 2.1254167556762695]

720 classes

In [5]:
import numpy as np
import matplotlib.pyplot as plt
from tensorflow import keras
from tensorflow.keras import layers
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow.keras import backend as K

In [6]:
y_c=[]
for i in range(720):
    la=np.zeros(720)
    la[i]=la[i]+1
    for j in range(25):
        y_c.append(la)

In [7]:
y_c=np.stack(y_c)
y_c=np.array(y_c,dtype=np.int32)

In [8]:
def common_sense_cla(y_true, y_pred):
    d = K.abs(K.argmax(y_true, axis=1) - K.argmax(y_pred, axis=1))
    d = tf.cast(d, tf.float32)
    return 0.5 * ((1 - (d // 360)) * (d) + (d // 360) * (720 - d))

In [9]:
X, X_test, y, y_test = train_test_split(data, y_c, test_size=0.2, random_state=42)
print(f"Train set: {X.shape}")
print(f"Test set: {X_test.shape}")

Train set: (14400, 150, 150)
Test set: (3600, 150, 150)


In [10]:
classification_model = keras.Sequential(
    [
        keras.Input(shape=(150,150,1)),
        layers.Conv2D(16, kernel_size=(3, 3), activation="relu",strides=1),
        layers.MaxPooling2D(pool_size=(2, 2)),
        layers.Conv2D(32, kernel_size=(3, 3), activation="relu",strides=2),
        layers.MaxPooling2D(pool_size=(2, 2)),
        layers.Conv2D(64, kernel_size=(3, 3), activation="relu",strides=1),
        layers.MaxPooling2D(pool_size=(2, 2)),
        layers.Flatten(),
        layers.Dense(64,activation=keras.layers.LeakyReLU(alpha=0.01)),
        layers.Dense(720,activation="softmax"),
    ]
)

classification_model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 148, 148, 16)      160       
                                                                 
 max_pooling2d (MaxPooling2D  (None, 74, 74, 16)       0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 36, 36, 32)        4640      
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 18, 18, 32)       0         
 2D)                                                             
                                                                 
 conv2d_2 (Conv2D)           (None, 16, 16, 64)        18496     
                                                                 
 max_pooling2d_2 (MaxPooling  (None, 8, 8, 64)         0

In [11]:
batch_size = 128
epochs = 20
classification_model.compile(loss="categorical_crossentropy", optimizer=tf.keras.optimizers.Adam(learning_rate=0.001,), metrics=["accuracy",common_sense_cla], run_eagerly=True)

In [12]:
classification_model.fit(X,y, batch_size=batch_size, epochs=epochs, validation_split=0.1)

Epoch 1/20
102/102 [==============================] - 9s 45ms/step - loss: 6.5805 - accuracy: 9.2593e-04 - common_sense_cla: 90.2356 - val_loss: 6.5812 - val_accuracy: 0.0000e+00 - val_common_sense_cla: 89.3837
Epoch 2/20
102/102 [==============================] - 5s 45ms/step - loss: 6.5793 - accuracy: 0.0018 - common_sense_cla: 89.8209 - val_loss: 6.5830 - val_accuracy: 0.0000e+00 - val_common_sense_cla: 89.5719
Epoch 3/20
102/102 [==============================] - 5s 45ms/step - loss: 6.5786 - accuracy: 0.0014 - common_sense_cla: 89.9103 - val_loss: 6.5849 - val_accuracy: 0.0000e+00 - val_common_sense_cla: 89.5719
Epoch 4/20
102/102 [==============================] - 5s 44ms/step - loss: 6.5781 - accuracy: 0.0017 - common_sense_cla: 89.8167 - val_loss: 6.5865 - val_accuracy: 0.0000e+00 - val_common_sense_cla: 90.6976
Epoch 5/20
102/102 [==============================] - 5s 44ms/step - loss: 6.5776 - accuracy: 0.0019 - common_sense_cla: 89.6294 - val_loss: 6.5883 - val_accuracy: 0.00

Multi-head models

## Task 3: Generative Models

In this task, we use MNIST dataset to train the generative models and obtain new figures. We directly call the Tensorflow API to download the dataset. However, the original dataset is also available on this website: <https://deepai.org/dataset/mnist>.

In [ ]:
import os
import re
import shutil
import numpy as np
import cv2
from tqdm import tqdm
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras.layers import Dense, Flatten, Conv2D, Conv2DTranspose, Reshape, Input

In [ ]:
# Download MNIST data
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.mnist.load_data()

# Preprocessing
X = np.concatenate((x_train, x_test), axis=0)
np.random.seed(42)
np.random.shuffle(X)
X = X.astype("float32") / 255.
X = X.reshape(X.shape + (1,))
print(f"The shape of dataset: {X.shape}")

In [ ]:
# Use modified grid_plot functions in the tutorial           
def grid_plot(images, epoch="", name="", n=3, save=False, scale=False):
    if scale:
        images = (images + 1) / 2.0
    for index in range(n * n):
        plt.subplot(n, n, 1 + index)
        plt.axis('off')
        plt.imshow(images[index])
    fig = plt.gcf()
    fig.suptitle(name + '  '+ str(epoch), fontsize=14)
    if save:
        filename = 'results/generated_plot_e%03d_f.png' % (epoch+1)
        plt.savefig(filename)
        plt.close()
    plt.show()

In [ ]:
np.random.seed(42)
grid_plot(X[np.random.randint(0, 1000, 9)], name="MNIST dataset (28 X 28 X 1)", n=3)

In [ ]:
# Use modified build_conv_net and build_deconv_net functions in the tutorial
def build_conv_net(in_shape, out_shape, out_activation='sigmoid'):
    """
    Build a basic convolutional network
    """
    in_put = Input(shape=in_shape)
    
    x = Conv2D(kernel_size=3, filters=32, padding='same', activation='relu')(in_put)
    x = Conv2D(kernel_size=3, filters=64, padding='same', activation='relu', strides=(2,2))(x)
    x = Conv2D(kernel_size=3, filters=64, padding='same', activation='relu')(x)
    x = Conv2D(kernel_size=3, filters=64, padding='same', activation='relu')(x)
    
    x = Flatten()(x)
    x = Dense(out_shape, activation=out_activation)(x)
    model = tf.keras.Model(in_put, x)
    model.summary()
    return model


def build_deconv_net(latent_dim, activation_out='sigmoid'):
    """
    Build a deconvolutional network for decoding/upscaling latent vectors

    When building the deconvolutional architecture, usually it is best to use the same layer sizes that 
    were used in the downsampling network and the Conv2DTranspose layers are used instead of Conv2D layers. 
    Using identical layers and hyperparameters ensures that the dimensionality of our output matches the
    shape of our input images. 
    """
    in_put = Input(shape=latent_dim)
    x = Dense(14 * 14 * 64)(in_put)
    x = Reshape((14, 14, 64))(x) # This matches the output size of the downsampling architecture
    x = Conv2DTranspose(32, 3, padding='same', activation='relu', strides=(2,2))(x)
    x = Conv2D(filters=1, kernel_size=3, activation=activation_out, padding='same')(x)
    model = tf.keras.Model(in_put, x)
    model.summary()
    return model

### Convolutional Autoencoder (CAE)

In [ ]:
# CAE
# Use the build_convolutional_autoencoder function in the tutorial
def build_convolutional_autoencoder(data_shape, latent_dim):
    encoder = build_conv_net(in_shape=data_shape, out_shape=latent_dim)
    decoder = build_deconv_net(latent_dim, activation_out='sigmoid')
    autoencoder = tf.keras.Sequential([encoder, decoder])
    autoencoder.compile(loss='binary_crossentropy', optimizer='adam')
    return autoencoder

# Defining the model dimensions and building it
image_size = X.shape[1:]
latent_dim = 32
num_filters = 64
cae = build_convolutional_autoencoder(image_size, latent_dim)

for epoch in range(0, 11):
    cae.fit(x=X, y=X, epochs=1, batch_size=64, verbose=0)
    print('\nEpoch: ', epoch)
    samples = X[:9]
    reconstructed = cae.predict(samples, verbose=0)
    grid_plot(samples, epoch, name='Original', n=3, save=False)
    grid_plot(reconstructed, epoch, name='Reconstructed', n=3, save=False)

### Variational Autoencoders (VAEs)

In [ ]:
# Use the Sampling class and the build_vae function in the tutorial
class Sampling(tf.keras.layers.Layer):
    """
    Custom layer for the variational autoencoder
    It takes two vectors as input - one for means and other for variances of the latent variables described by a multimodal gaussian
    Its output is a latent vector randomly sampled from this distribution
    """
    def call(self, inputs):
        z_mean, z_var = inputs
        batch = tf.shape(z_mean)[0]
        dim = tf.shape(z_mean)[1]
        epsilon = tf.keras.backend.random_normal(shape=(batch, dim))
        return z_mean + tf.exp(0.5 * z_var) * epsilon

    
def build_vae(data_shape, latent_dim, filters=128):

    # Building the encoder - starts with a simple downsampling convolutional network  
    encoder = build_conv_net(data_shape, latent_dim*2)
    
    # Adding special sampling layer that uses the reparametrization trick 
    z_mean = Dense(latent_dim)(encoder.output)
    z_var = Dense(latent_dim)(encoder.output)
    z = Sampling()([z_mean, z_var])
    
    # Connecting the two encoder parts
    encoder = tf.keras.Model(inputs=encoder.input, outputs=z)

    # Defining the decoder which is a regular upsampling deconvolutional network
    decoder = build_deconv_net(latent_dim, activation_out='sigmoid')
    vae = tf.keras.Model(inputs=encoder.input, outputs=decoder(z))
    
    # Adding the special loss term
    kl_loss = -0.5 * tf.reduce_sum(z_var - tf.square(z_mean) - tf.exp(z_var) + 1)
    vae.add_loss(kl_loss/tf.cast(tf.keras.backend.prod(data_shape), tf.float32))

    vae.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=1e-3), loss='binary_crossentropy')

    return encoder, decoder, vae

In [ ]:
# Training the VAE model
latent_dim = 32
encoder, decoder, vae = build_vae(X.shape[1:], latent_dim)

# Generate random vectors that we will use to sample our latent space
for epoch in range(0, 20):
    vae.fit(x=X, y=X, epochs=1, batch_size=16, verbose=1)
    latent_vectors = np.random.randn(9, latent_dim)
    images = decoder(latent_vectors)
    grid_plot(images, epoch, name='VAE generated images (randomly sampled from the latent space)', n=3, save=False)

In [ ]:
latent_dim = 32
for i in range(-2, 3):
    np.random.seed(42)
    latent_vectors = np.tile(np.random.randn(latent_dim), (7, 1))
    latent_vectors[:, 9] = np.linspace(-5, 5, num=7)
    latent_vectors[:, 26] = i * np.ones(7)

    images = decoder(latent_vectors)
    plt.figure(figsize = (15, 1.5))
    for index in range(7):
        plt.subplot(1, 7, index + 1)
        plt.axis('off')
        plt.imshow(images[index], aspect='auto')
    plt.show()

### Generative Adversarial Networks (GANs)

In [ ]:
# Use build_gan, run_generator, get_batch, train_gan function in the tutorial
def build_gan(data_shape, latent_dim, lr=0.0002, beta_1=0.5):
    optimizer = tf.optimizers.Adam(learning_rate=lr, beta_1=beta_1)

    # Usually the GAN generator has tanh activation function in the output layer
    generator = build_deconv_net(latent_dim, activation_out='tanh')
    
    # Build and compile the discriminator
    discriminator = build_conv_net(in_shape=data_shape, out_shape=1) # Single output for binary classification
    discriminator.compile(loss='binary_crossentropy', optimizer=optimizer)
    
    # End-to-end GAN model for training the generator
    discriminator.trainable = False
    true_fake_prediction = discriminator(generator.output)
    GAN = tf.keras.Model(inputs=generator.input, outputs=true_fake_prediction)
    GAN = tf.keras.models.Sequential([generator, discriminator])
    GAN.compile(loss='binary_crossentropy', optimizer=optimizer)
    
    return discriminator, generator, GAN


def run_generator(generator, n_samples=100):
    """
    Run the generator model and generate n samples of synthetic images using random latent vectors
    """
    latent_dim = generator.layers[0].input_shape[-1]
    generator_input = np.random.randn(n_samples, latent_dim[1])

    return generator.predict(generator_input)
    

def get_batch(generator, dataset, batch_size=64):
    """
    Gets a single batch of samples (X) and labels (y) for the training the discriminator.
    One half from the real dataset (labeled as 1s), the other created by the generator model (labeled as 0s).
    """
    batch_size //= 2 # Split evenly among fake and real samples

    fake_data = run_generator(generator, n_samples=batch_size)
    real_data = dataset[np.random.randint(0, dataset.shape[0], batch_size)]

    X = np.concatenate([fake_data, real_data], axis=0)
    y = np.concatenate([np.zeros([batch_size, 1]), np.ones([batch_size, 1])], axis=0)

    return X, y


def train_gan(generator, discriminator, gan, dataset, latent_dim, n_epochs=20, batch_size=64):

    batches_per_epoch = int(dataset.shape[0] / batch_size / 2)
    for epoch in range(n_epochs):
        for batch in range(batches_per_epoch):
            
            # 1) Train discriminator both on real and synthesized images
            X, y = get_batch(generator, dataset, batch_size=batch_size)
            discriminator_loss = discriminator.train_on_batch(X, y)

            # 2) Train generator (note that now the label of synthetic images is reversed to 1)
            X_gan = np.random.randn(batch_size, latent_dim)
            y_gan = np.ones([batch_size, 1])
            generator_loss = gan.train_on_batch(X_gan, y_gan)
            
        noise = np.random.randn(16, latent_dim)
        images = generator.predict(noise)
        grid_plot(images, epoch, name='GAN generated images', n=3, save=False, scale=True)

In [ ]:
# Build and train the model (need around 10 epochs to start seeing some results)
latent_dim = 256
discriminator, generator, gan = build_gan(X.shape[1:], latent_dim)

train_gan(generator, discriminator, gan, X, latent_dim, n_epochs=20)

In [ ]:
latent_dim = 256

np.random.seed(13)
latent_vectors = np.tile(np.random.randn(latent_dim), (7, 1))
latent_vectors[:, 0] = np.linspace(-30, 30, num=7)

images = generator(latent_vectors)
plt.figure(figsize = (15, 1.5))
for index in range(7):
    plt.subplot(1, 7, index + 1)
    plt.axis('off')
    plt.imshow(images[index], aspect='auto')
plt.show()

In [ ]:
latent_dim = 256

np.random.seed(13)
latent_vectors = np.tile(np.random.randn(latent_dim), (7, 1))
latent_vectors[:, 119] = np.linspace(-30, 30, num=7)

images = generator(latent_vectors)
plt.figure(figsize = (15, 1.5))
for index in range(7):
    plt.subplot(1, 7, index + 1)
    plt.axis('off')
    plt.imshow(images[index], aspect='auto')
plt.show()

In [ ]:
latent_dim = 256

np.random.seed(13)
latent_vectors = np.tile(np.random.randn(latent_dim), (7, 1))
latent_vectors[:, 125] = np.linspace(0, 30, num=7)

images = generator(latent_vectors)
plt.figure(figsize = (15, 1.5))
for index in range(7):
    plt.subplot(1, 7, index + 1)
    plt.axis('off')
    plt.imshow(images[index], aspect='auto')
plt.show()